In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np

import sys
import os

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *

import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import mxnet as mx

from joblib import Parallel, delayed
import time

In [ ]:
# labels =  ['BackG', '5N', '7n', '7N', '12N', 'Gr', 'LVe', 'Pn', 'SuVe', 'VLL']

labels = ['BackG', '5N', '7n', '7N', '12N', 'Pn', 'VLL', 
          '6N', 'Amb', 'R', 'Tz', 'RtTg', 'LRt', 'LC', 'AP', 'sp5']

label_dict = dict([(l,i) for i, l in enumerate(labels)])

# label_dict = dict([(l,i) for i, l in enumerate(labels)] + \
#                   zip(other_labels, range(len(labels), len(labels)+len(other_labels))))

In [ ]:
patches_rootdir = '/home/yuncong/CSHL_data_patches/'
model_dir = '/home/yuncong/mxnet_models/'

In [ ]:
batch_size = 32

In [ ]:
# model_name='Inception_BN'
# model_iteration = 39
# init_model = mx.model.FeedForward.load(os.path.join(model_dir, model_name), 
#                                      model_iteration, 
#                                      ctx=mx.gpu(),
#                                     numpy_batch_size=batch_size)

# arg_params = init_model.arg_params.copy()
# arg_params.pop('fc_bias');
# arg_params.pop('fc_weight');

In [ ]:
model_name='inception-stage1'
model_iteration = 10
init_model = mx.model.FeedForward.load(os.path.join(model_dir, model_name), 
                                     model_iteration, 
                                     ctx=mx.gpu(),
                                    numpy_batch_size=batch_size)


arg_params = init_model.arg_params.copy()
arg_params.pop('fullc_bias');
arg_params.pop('fullc_weight');

In [ ]:
n_class = len(labels)
internals = init_model.symbol.get_internals()
flatten_output = internals['flatten_output']
fc = mx.symbol.FullyConnected(data=flatten_output, name='fullc', num_hidden=n_class)
softmax = mx.symbol.SoftmaxOutput(data=fc, name='softmax')

In [ ]:
mean_img = mx.nd.load(os.path.join(model_dir, 'mean_224.nd'))['mean_img'].asnumpy()

In [ ]:
from IPython.html.widgets import FloatProgress
from IPython.display import display

progress_bar = FloatProgress(min=0, max=np.inf)

In [ ]:
patches_allClasses = []
patchLabels_allClasses = []

In [ ]:
for stack in ['MD589']:

    dm = DataManager(stack=stack)
    stack_has_annotation = True

    if stack_has_annotation:
        table_filepath = os.path.join(patches_rootdir, '%(stack)s_indices_allLandmarks_allSections.h5'%{'stack':stack})
        indices_allLandmarks_allSections = pd.read_hdf(table_filepath, 'indices_allLandmarks_allSections')
        grid_parameters = pd.read_hdf(table_filepath, 'grid_parameters')

    first_bs_sec, last_bs_sec = section_range_lookup[stack]
    first_detect_sec, last_detect_sec = detect_bbox_range_lookup[stack]

    progress_bar.min = first_detect_sec
    progress_bar.max = last_detect_sec
    display(progress_bar)
    
    for sec in range(first_detect_sec, last_detect_sec+1):
#     for sec in range(first_detect_sec, first_detect_sec+10):
#     for sec in range(first_detect_sec, first_detect_sec+1):
        
        if sec not in indices_allLandmarks_allSections.columns:
            continue

        progress_bar.value = sec
#         print sec

        ## define grid, generate patches

    #     t = time.time()
        dm.set_slice(sec)
        dm._load_image(['rgb-jpg'])

        patch_size, stride, w, h = grid_parameters.tolist()
        half_size = patch_size/2

        ys, xs = np.meshgrid(np.arange(half_size, h-half_size, stride), np.arange(half_size, w-half_size, stride),
                         indexing='xy')

        sample_locations = np.c_[xs.flat, ys.flat]


        q = indices_allLandmarks_allSections[sec].dropna()
        if len(q.index) == 0:
            continue

        for label_ind, label in enumerate(labels):

            print label

            if label == 'BackG':
                w = [q[l] for l in q.index if l.endswith('surround')]
                if len(w) == 0:
                    continue
                else:
                    indices_roi = np.concatenate(w)
            else:
                if label in q.index:
                    indices_roi = q[label]
                else:
                    continue

            n2 = len(indices_roi)
            print n2, 'samples'

            num_sample_each_polygon = 30
            indices_roi = np.random.choice(indices_roi, min(num_sample_each_polygon, n2), replace=False)

            n = len(indices_roi)
            print n, 'used samples'

            sample_locations_roi = sample_locations[indices_roi]

            patches2 = np.asarray([dm.image_rgb_jpg[y-half_size:y+half_size, x-half_size:x+half_size]
                                  for x, y in sample_locations_roi])

            patches = np.rollaxis(patches2, 3, 1)
            patches = patches - mean_img

            patches_allClasses.append(patches.copy())

            patch_labels = label_ind * np.ones((n, ), np.int)
            patchLabels_allClasses.append(patch_labels)

            del patches, patches2, sample_locations_roi

        del sample_locations

    #     break
    #     sys.stderr.write('generating patches: %.2f seconds\n' % (time.time() - t))


In [ ]:
patches_allClasses_arr = np.concatenate(patches_allClasses)
patchLabels_allClasses_arr = np.concatenate(patchLabels_allClasses)

In [ ]:
len(patchLabels_allClasses_arr)

In [ ]:
patches_allClasses_eval = []
patchLabels_allClasses_eval = []

In [ ]:
for stack in ['MD594']:

    dm = DataManager(stack=stack)
    stack_has_annotation = True

    if stack_has_annotation:
        table_filepath = os.path.join(patches_rootdir, '%(stack)s_indices_allLandmarks_allSections.h5'%{'stack':stack})
        indices_allLandmarks_allSections = pd.read_hdf(table_filepath, 'indices_allLandmarks_allSections')
        grid_parameters = pd.read_hdf(table_filepath, 'grid_parameters')

    first_bs_sec, last_bs_sec = section_range_lookup[stack]
    first_detect_sec, last_detect_sec = detect_bbox_range_lookup[stack]

    progress_bar.min = first_detect_sec
    progress_bar.max = last_detect_sec
    display(progress_bar)
    
    for sec in range(first_detect_sec, last_detect_sec+1):
        
        if sec not in indices_allLandmarks_allSections.columns:
            continue

        progress_bar.value = sec
#         print sec

        ## define grid, generate patches

    #     t = time.time()
        dm.set_slice(sec)
        dm._load_image(['rgb-jpg'])

        patch_size, stride, w, h = grid_parameters.tolist()
        half_size = patch_size/2

        ys, xs = np.meshgrid(np.arange(half_size, h-half_size, stride), np.arange(half_size, w-half_size, stride),
                         indexing='xy')

        sample_locations = np.c_[xs.flat, ys.flat]


        q = indices_allLandmarks_allSections[sec].dropna()
        if len(q.index) == 0:
            continue

        for label_ind, label in enumerate(labels):

            print label

            if label == 'BackG':
                w = [q[l] for l in q.index if l.endswith('surround')]
                if len(w) == 0:
                    continue
                else:
                    indices_roi = np.concatenate(w)
            else:
                if label in q.index:
                    indices_roi = q[label]
                else:
                    continue

            n2 = len(indices_roi)
            print n2, 'samples'

            num_sample_each_polygon = 30
            indices_roi = np.random.choice(indices_roi, min(num_sample_each_polygon, n2), replace=False)

            n = len(indices_roi)
            print n, 'used samples'

            sample_locations_roi = sample_locations[indices_roi]

            patches2 = np.asarray([dm.image_rgb_jpg[y-half_size:y+half_size, x-half_size:x+half_size]
                                  for x, y in sample_locations_roi])

            patches = np.rollaxis(patches2, 3, 1)
            patches = patches - mean_img

            patches_allClasses_eval.append(patches.copy())

            patch_labels = label_ind * np.ones((n, ), np.int)
            patchLabels_allClasses_eval.append(patch_labels)

            del patches, patches2, sample_locations_roi

        del sample_locations

    #     break
    #     sys.stderr.write('generating patches: %.2f seconds\n' % (time.time() - t))


In [ ]:
len(patches_allClasses_eval)

In [ ]:
patches_allClasses_eval_arr = np.concatenate(patches_allClasses_eval)
patchLabels_allClasses_eval_arr = np.concatenate(patchLabels_allClasses_eval)

In [ ]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [ ]:
n_epoch = 10
l_rate = 0.001
# checkpoint_prefix = os.path.join(model_dir, 'experiment0227')
checkpoint_prefix = os.path.join(model_dir, 'experiment0317')

opt = mx.optimizer.SGD(learning_rate=l_rate)

net = mx.model.FeedForward(ctx=mx.gpu(), 
#                            symbol=init_model.symbol, 
                           symbol=softmax, 
                           num_epoch=n_epoch, optimizer=opt,
                           arg_params = arg_params, 
                           aux_params = init_model.aux_params,
                           allow_extra_params = True,
                           numpy_batch_size = batch_size)

In [ ]:
batch_size = 32

train_iter = mx.io.NDArrayIter(
    patches_allClasses_arr, 
    patchLabels_allClasses_arr,
    batch_size = batch_size,
    shuffle=True
)

# eval_iter = mx.io.NDArrayIter(
#     patches_allClasses_eval_arr, 
#     patchLabels_allClasses_eval_arr,
#     batch_size = batch_size,
#     shuffle=True
# )

t = time.time()

# net.fit(train_iter, eval_data=eval_iter,
#         batch_end_callback=mx.callback.Speedometer(batch_size, 30),
#         epoch_end_callback=mx.callback.do_checkpoint(checkpoint_prefix))

net.fit(train_iter,
        batch_end_callback=mx.callback.Speedometer(batch_size, 30),
        epoch_end_callback=mx.callback.do_checkpoint(checkpoint_prefix))

net.save(checkpoint_prefix)

print time.time() - t

# 100 samples/sec


In [ ]:
q = np.where(patchLabels_allClasses == label_dict['7N'])[0]

for p in patches_allClasses[q]:
    plt.imshow(np.rollaxis(p + mean_img, 0, 3).astype(np.uint8));
    plt.show();